In [1]:
from ipywidgets import interact, interact_manual, widgets, Label
import numpy as np
import functools
from time import time, sleep
from os import system, listdir
import h5py

In [2]:
s_url='https://github.com/patrickhaddadteaching/TRNG_ex2/raw/main/data/data_for_ex2.zip'
a=system('wget %s>test.log'%s_url)
#a=system('cp zips/data_for_ex2.zip . >test.log')
a=system('unzip data_for_ex2.zip > test.log')
system('rm -Rf data_for_ex2.zip')

0

In [3]:
hdf5_file = 'data_for_ex2.h5'
data_file = h5py.File(hdf5_file,'r')
v_ratio_period_sample_for_interact=np.array(data_file['parameters/v_ratio_period_sample'][0,:],dtype=np.uint32)
v_jitter_for_interact=data_file['parameters/v_jitter'][0,:]
v_raw_rng_image=data_file['images/rng_image'][0,:]
m_raw_bits_from_one_RO=data_file['raw_bits/m_raw_bits_from_one_RO'][:]
data_file.close()
system('rm -Rf %s'%hdf5_file)
nb_test_per_corner=m_raw_bits_from_one_RO.shape[3]
nb_bits=m_raw_bits_from_one_RO.shape[4]
nb_osc=m_raw_bits_from_one_RO.shape[0]

In [4]:
width_png=547
height_png=400

In [5]:
image_height=250
image_width=int(width_png*image_height/height_png)
raw_bytes_width=100
tests_results_width=300

In [6]:
def run_fips_tests(v_bits):
    len_v_bits=v_bits.shape[0]
    nb_nibble=int((v_bits.shape[0])/4)
    v_nibble=np.zeros((nb_nibble,),dtype=np.uint8)
    for i in range(4):
        v_nibble+=np.array(v_bits[i:4*nb_nibble:4],dtype=np.uint8)*(2**i)

    T1=v_bits.sum()

    T2=0
    for j in range(16):
        fj=(v_nibble==j).sum()
        T2+=((fj-5000.0/16)**2)/(5000.0/16)

    nb_lambda=32
    m_T3=np.zeros((nb_lambda,2),dtype=np.float64)
    for lambda_i in range(nb_lambda):
        word_size_in_bits=lambda_i+3

        nb_words=int(len_v_bits-word_size_in_bits)
        v_words=np.zeros((nb_words,),dtype=np.uint64)
        for i in range(word_size_in_bits):
            v_words+=np.array(v_bits[i:i+nb_words],dtype=np.uint8)*(2**i)    

        patern_0=2**(word_size_in_bits-1)+1
        patern_1=(2**(word_size_in_bits)-2**(word_size_in_bits-1)-2)
        m_T3[lambda_i,0]=(v_words==patern_0).sum()
        m_T3[lambda_i,1]=(v_words==patern_1).sum()

    lambda_T4=34
    v_sum=np.zeros((len_v_bits-lambda_T4,),dtype=np.uint64)
    for i in range(lambda_T4):
        v_sum+=v_bits[i:len_v_bits-lambda_T4+i]

    T4_PS=((v_sum==0) | (v_sum==lambda_T4)).sum()

    v_T5=np.zeros((5000,),dtype=np.uint64)
    for tau in range(5000):
        v_T5[tau]=(v_bits[:5000]^v_bits[tau+1:tau+5000+1]).sum()
    return (T1,T2,[[m_T3[0,0],m_T3[0,1]],[m_T3[1,0],m_T3[1,1]],[m_T3[2,0],m_T3[2,1]],[m_T3[3,0],m_T3[3,1]],[m_T3[4,0],m_T3[4,1]],[m_T3[5:,0].sum(),m_T3[5:,1].sum()]],T4_PS,v_T5)


s_label_test_init='<p><b>Procedure A of AIS31</b></p>'
s_label_test_init='%s<p><b>*****************************************</b></p>'%(s_label_test_init)   

def log_fips_tests(T1,T2,M_T3,T4_PS,v_T5):
    s_color_code_succeed='00FF00'
    s_color_code_failed='FF0000'  
    
    s_label_test=s_label_test_init

    if abs(T1-nb_bits/2)<346:
        s_label_test='%s<p style="color:#00FF00";><b>Monobit Test Succeed</b></p>'%(s_label_test)
    else:
        s_label_test='%s<p style="color:#FF0000";><b>Monobit Test Failed</b></p>'%(s_label_test)

    if (T2<57.4)&(T2>1.03):
        s_label_test='%s<p style="color:#00FF00";><b>Poker Test Succeed</b></p>'%(s_label_test)        
    else:
        s_label_test='%s<p style="color:#FF0000";><b>Poker Test Failed</b></p>'%(s_label_test)

    m_T3_bounds=[[2267,2733],[1079,1421],[502,748],[223,402],[90,223],[90,223]]
    cnt_error=0
    for b in range(2):
        for i in range(6):
            if (M_T3[i][b]<m_T3_bounds[i][1])&(M_T3[i][b]>m_T3_bounds[i][0]):
                cnt_error+=1
                
    if (cnt_error>0):
        s_label_test='%s<p style="color:#00FF00";><b>Runs Test Succeed</b></p>'%(s_label_test)     
    else:
        s_label_test='%s<p style="color:#FF0000";><b>Runs Test Failed</b></p>'%(s_label_test)

        
    if (T4_PS==False):
        s_label_test='%s<p style="color:#00FF00";><b>Long Runs Test Succeed</b></p>'%(s_label_test)        
    else:
        s_label_test='%s<p style="color:#FF0000";><b>Long Runs Test Failed</b></p>'%(s_label_test)
        
    T5=v_T5.max()
    if (T5<2674)&(T5>2326):
        s_label_test='%s<p style="color:#00FF00";><b>Autocorrelation Test Succeed</b></p>'%(s_label_test)
    else:
        s_label_test='%s<p style="color:#FF0000";><b>Autocorrelation Test Failed</b></p>'%(s_label_test)
    
    return s_label_test

In [7]:

sample_period_value_widgets=widgets.SelectionSlider(description=r'\(D\)',options=list(v_ratio_period_sample_for_interact),layout=widgets.Layout(width='%dpx'%(image_height+raw_bytes_width)),readout_format='.1f')
m_value_widgets=widgets.SelectionSlider(description=r'\(m\)',options=list(np.arange(nb_osc,dtype=np.uint8)+1),layout=widgets.Layout(width='%dpx'%(image_height+raw_bytes_width)))
std_dev_value_widgets=widgets.SelectionSlider(description=r'\(\frac{\sigma_{tot}}{T_{1}}\)',options=list(v_jitter_for_interact),layout=widgets.Layout(width='%dpx'%(image_height+raw_bytes_width)))

rng_image=widgets.Image(value=v_raw_rng_image.tobytes(), format='png',layout=widgets.Layout(height='%dpx'%(image_height),width='%dpx'%image_width))
tests_results=widgets.HTML(value=s_label_test_init,placeholder='',description='',disabled=True,layout=widgets.Layout(height='%dpx'%(image_height),width='auto'))
space_for_tests_results=widgets.HTML(value='',placeholder='',description='',disabled=True,layout=widgets.Layout(height='%dpx'%image_height,width='%dpx'%(raw_bytes_width/6) ))
raw_bytes=widgets.Textarea(value='',rows=5000,placeholder='',description='',disabled=True,layout=widgets.Layout(height='%dpx'%(image_height),width='%dpx'%(image_width/4)))

float_bar=widgets.FloatProgress(value=0.0,min=0.0, max=100.0,description='',bar_style='info',style={'bar_color': '#ff0000'},orientation='horizontal',layout=widgets.Layout(width='%dpx'%tests_results_width))
val_button=widgets.Button(description = 'GENERATE RANDOM VALUES',layout=widgets.Layout(width='%dpx'%tests_results_width))
val_button.style.button_color = 'gray'

def generate_rnd(b):
    raw_bytes.value=''
    chosen_sample_period=sample_period_value_widgets.value
    chosen_jitter=std_dev_value_widgets.value
    chosen_m=m_value_widgets.value    
    
    index_sample_period=np.where(chosen_sample_period==v_ratio_period_sample_for_interact)[0][0]
    index_chosen_jitter=np.where(chosen_jitter==v_jitter_for_interact)[0][0]

    v_raw_bits=np.zeros((nb_bits,),dtype=np.bool)
    for i in range(chosen_m):
        tmp_rand=np.random.randint(0,nb_test_per_corner,dtype=np.uint32)
        v_raw_bits^=m_raw_bits_from_one_RO[i-1,index_chosen_jitter,index_sample_period,tmp_rand,:]
        
    nb_nibble=int((nb_bits)/4)
    v_nibble=np.zeros((nb_nibble,),dtype=np.uint8)
    for i in range(4):
        v_nibble+=np.array(v_raw_bits[i:4*nb_nibble:4],dtype=np.uint8)*(2**i)
    s_line=''
    for i in range(nb_nibble):
        s_line='%s%X'%(s_line,v_nibble[i])
    raw_bytes.value=s_line   

    (T1_o,T2_o,M_T3_o,T4_PS_o,v_T5_o)=run_fips_tests(v_raw_bits)
    tests_results.value=log_fips_tests(T1_o,T2_o,M_T3_o,T4_PS_o,v_T5_o)
    
    float_bar.value=0
    time_to_sleep=0.01
    for i in range(100):
        sleep(time_to_sleep)
        float_bar.value=(i+1)
val_button.on_click(generate_rnd, remove=False) 
v_box_top=widgets.VBox([widgets.HBox([rng_image,raw_bytes,space_for_tests_results,tests_results]),widgets.VBox([widgets.HBox([widgets.VBox([m_value_widgets,std_dev_value_widgets,sample_period_value_widgets]),widgets.VBox([val_button,float_bar])])])])
display(v_box_top)

['.ipynb_checkpoints', 'Exercise_1.ipynb', 'Exercise_2.ipynb', 'Exercise_5.ipynb', 'figures', 'generate_constants_entropies_several_osc_on one_hdf5.ipynb', 'generate_constants_one_osc_on one_hdf5.ipynb', 'generate_constants_one_osc_on one_hdf5.py', 'generate_constants_several_osc_on one_hdf5.ipynb', 'generate_constants_several_osc_on one_hdf5.py', 'randomvalues.txt', 'sampling_proba_entropy_2osc.R', 'test.log', 'test.txt', 'tmp_ex1.svg', 'tmp_ex2.svg', 'TRNG_ex1_nb.ipynb', 'TRNG_ex1_nb_v0.1.ipynb', 'TRNG_ex2_nb.ipynb', 'TRNG_ex2_nb_v0.1.ipynb', 'TRNG_ex3_nb.ipynb', 'TRNG_ex4_nb.ipynb', 'TRNG_ex5_nb.ipynb', 'TRNG_ex5_nb_v0.1.ipynb', 'Untitled.ipynb', 'zips']
['.ipynb_checkpoints', 'Exercise_1.ipynb', 'Exercise_2.ipynb', 'Exercise_5.ipynb', 'figures', 'generate_constants_entropies_several_osc_on one_hdf5.ipynb', 'generate_constants_one_osc_on one_hdf5.ipynb', 'generate_constants_one_osc_on one_hdf5.py', 'generate_constants_several_osc_on one_hdf5.ipynb', 'generate_constants_several_osc_o